In [1]:
import torch

\begin{equation}
    u_{i,j} =
    \begin{bmatrix}
      \frac{\partial u_1}{\partial x_1} & 
        \frac{\partial u_1}{\partial x_2} & 
        \frac{\partial u_1}{\partial x_3} \\[1ex] % <-- 1ex more space between rows of matrix
      \frac{\partial u_2}{\partial x_1} & 
        \frac{\partial u_2}{\partial x_2} & 
        \frac{\partial u_2}{\partial x_3} \\[1ex]
      \frac{\partial u_3}{\partial x_1} & 
        \frac{\partial u_3}{\partial x_2} & 
        \frac{\partial u_3}{\partial x_3}
    \end{bmatrix}
\end{equation}

## The `autograd` package

This package provides automatic differentiation for Tensors.

All `torch.Tensor` objects have an attribute called `.requires_grad`. When we set it as `True`, torch starts to track all the operations in order to compute its gradient. When all the computation is done, we can call the `.backward()` method, and the gradients will be computed and accumulated into `.grad` attribute.

If we want a `torch.Tensor` to stop tracking operations, we should call `.detach()`

Each `torch.Tensor` that uses `.requires_grad`, uses its `.grad_fn` to store an auxiliar function that helps `autograd` to compute gradients.

### 1. Simple example

Let $f$ be a differentiable function as, for example, $f = \frac{1}{10} (x - 2)^2$. This way, $\frac{\partial f}{\partial x} = 5 (x - 2)$.

Now, lets suppose that $x = 1$. The gradient at this point shuold be $\frac{\partial f}{\partial x} = -5$.

In [2]:
x = torch.ones(1, requires_grad=True)
print(x)

tensor([1.], requires_grad=True)


In [3]:
y = 1/10 * (x) **2
print(y, y.grad_fn)

tensor([0.1000], grad_fn=<MulBackward0>) <MulBackward0 object at 0x125c76b70>


In [4]:
y.backward()

In [5]:
print("The gradiend should be 1/5, and the result is:", x.grad)

The gradiend should be 1/5, and the result is: tensor([0.2000])
